# 训练过程

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)
batch_size = 100
learning_rate = 0.8
learning_rate_decay = 0.999
max_steps = 30000

#更改前向传播算法的定义,将得到权重参数和偏执参数的过程封装到了一个函数中
def hidden_layer(input_tensor,regularizer,name):
#要多多体会使用变量空间来管理变量的方便性。使用get_variable()函数会在训练神经
#网络时创建这些变量而在测试过程中通过保存的模型加载这些变量的取值，在测试过程中
#可以在加载变量时将滑动平均变量重命名，这样就会在测试过程中使用变量的滑动平均值
    with tf.variable_scope("hidden_layer"):
        weights = tf.get_variable("weights", [784, 500],
                         initializer=tf.truncated_normal_initializer(stddev=0.1))

        #如果调用该函数时传入了正则化的方法 ，那么在这里将参数求
        if regularizer!=None:
            tf.add_to_collection("losses",regularizer(weights))
        biases = tf.get_variable("biases", [500], initializer=tf.constant_initializer(0.0))
        hidden_layer = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)

    with tf.variable_scope("hidden_layer_output"):
        weights = tf.get_variable("weights", [500, 10],
                                  initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer!=None:
            tf.add_to_collection("losses",regularizer(weights))
        biases = tf.get_variable("biases", [10], initializer=tf.constant_initializer(0.0))
        hidden_layer_output = tf.matmul(hidden_layer, weights) + biases
    return hidden_layer_output

#定义输出输出的部分没变
x = tf.placeholder(tf.float32, [None,784],name="x-input")
y_ = tf.placeholder(tf.float32, [None,10],name="y-output")

#定义L2正则化的办法被提前
regularizer = tf.contrib.layers.l2_regularizer(0.0001)

#将L2正则化的办法传入到hidden_layer()函数中
y = hidden_layer(x,regularizer,name="y")

training_step = tf.Variable(0,trainable=False)
averages_class = tf.train.ExponentialMovingAverage(0.99,training_step)
averages_op = averages_class.apply(tf.trainable_variables())

#不再定义average_y，因为average_y只在比较正确率时有用，
#在模型保存的程序中我们只输出损失
#average_y = hidden_layer(x,averages_class,name="average_y",reuse=True)

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,
                                                 labels=tf.argmax(y_, 1)) #tf.argmax()
#计算总损失
loss = tf.reduce_mean(cross_entropy)+tf.add_n(tf.get_collection("losses"))

laerning_rate = tf.train.exponential_decay(learning_rate,training_step,
                mnist.train.num_examples/batch_size,learning_rate_decay)
train_step= tf.train.GradientDescentOptimizer(learning_rate).\
                                minimize(loss,global_step=training_step)

#也可以采用train_op = tf.group(train_step,averages_op)的形式
with tf.control_dependencies([train_step, averages_op]):
    train_op = tf.no_op(name="train")

#初始化Saver持久化类
saver=tf.train.Saver()
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    #进行30000轮到训练
    for i in range(max_steps):
        x_train, y_train = mnist.train.next_batch(batch_size)
        _, loss_value, step = sess.run([train_op, loss, training_step],
                                          feed_dict={x: x_train, y_: y_train})

        #每隔1000轮训练就输出当前训练batch上的损失函数大小，并保存一次模型
        if i % 1000 == 0:
            print("After %d training step(s), loss on training batch is "
                                                    "%g." % (step, loss_value))
            #保存模型的时候给出了global_step参数，这样可以让每个模型文件都添加
            #代表了训练轮数的后缀，这样做的原因是方便检索
            saver.save(sess, "mnist_model/mnist_model.ckpt",global_step=training_step)

W1201 11:33:49.798818 140421678028608 deprecation.py:323] From <ipython-input-1-39a51f8e8617>:3: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1201 11:33:49.800527 140421678028608 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1201 11:33:49.801787 140421678028608 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed 

Extracting MNIST_data/train-images-idx3-ubyte.gz


W1201 11:33:50.042194 140421678028608 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W1201 11:33:50.045698 140421678028608 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W1201 11:33:50.095971 140421678028608 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


W1201 11:33:50.342240 140421678028608 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/moving_averages.py:433: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


After 1 training step(s), loss on training batch is 3.10689.
After 1001 training step(s), loss on training batch is 0.251102.
After 2001 training step(s), loss on training batch is 0.166322.
After 3001 training step(s), loss on training batch is 0.131995.
After 4001 training step(s), loss on training batch is 0.11263.


W1201 11:34:27.204476 140421678028608 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


After 5001 training step(s), loss on training batch is 0.115287.
After 6001 training step(s), loss on training batch is 0.0956464.
After 7001 training step(s), loss on training batch is 0.0801559.
After 8001 training step(s), loss on training batch is 0.0718334.
After 9001 training step(s), loss on training batch is 0.0667971.
After 10001 training step(s), loss on training batch is 0.0624781.
After 11001 training step(s), loss on training batch is 0.060866.
After 12001 training step(s), loss on training batch is 0.0541651.
After 13001 training step(s), loss on training batch is 0.0514227.
After 14001 training step(s), loss on training batch is 0.0548598.
After 15001 training step(s), loss on training batch is 0.0456667.
After 16001 training step(s), loss on training batch is 0.044763.
After 17001 training step(s), loss on training batch is 0.0460693.
After 18001 training step(s), loss on training batch is 0.0438037.
After 19001 training step(s), loss on training batch is 0.037962.
Afte

# 测试过程

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data", one_hot=True)


# 定义相同的前向传播过程，要保持命名空间和变量名的一致
def hidden_layer(input_tensor, regularizer, name):
    with tf.variable_scope("hidden_layer"):
        weights = tf.get_variable("weights", [784, 500],
                                  initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None:
            tf.add_to_collection("losses", regularizer(weights))
        biases = tf.get_variable("biases", [500], initializer=tf.constant_initializer(0.0))
        hidden_layer = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)

    with tf.variable_scope("hidden_layer_output"):
        weights = tf.get_variable("weights", [500, 10],
                                  initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer != None:
            tf.add_to_collection("losses", regularizer(weights))
        biases = tf.get_variable("biases", [10], initializer=tf.constant_initializer(0.0))
        hidden_layer_output = tf.matmul(hidden_layer, weights) + biases
    return hidden_layer_output


x = tf.placeholder(tf.float32, [None, 784], name="x-input")
y_ = tf.placeholder(tf.float32, [None, 10], name="y-input")

# 因为测试时不必关注正则化损失的值，所以不会传入正则化的办法
y = hidden_layer(x, None, name="y")

# 计算正确率的过程也基本和第六章的样例一致
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
variable_averages = tf.train.ExponentialMovingAverage(0.99)

# 通过变量重命名的方式加载模型，这里使用了滑动平均类提供的variables_to_restore()
# 于是就免去了在前向传播过程中调用求解滑动平均的函数来获取滑动平均值的过程
saver = tf.train.Saver(variable_averages.variables_to_restore())

with tf.Session() as sess:
    validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
    test_feed = {x: mnist.test.images, y_: mnist.test.labels}

    # get_checkpoint_state()函数会通过checkpoint文件自动找到目录中最新模型的文件名
    # 函数原型get_checkpoint_state(checkpoint_dir,latest_filename)
    ckpt = tf.train.get_checkpoint_state("mnist_model/")

    # 加载模型
    saver.restore(sess, ckpt.model_checkpoint_path)

    # 通过文件名得到模型保存时迭代的轮数
    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
    print("The latest ckpt is mnist_model.ckpt-%s" % (global_step))
    # 输出The latest ckpt is mnist_model.ckpt-29001

    # 计算在验证数据集上的准确率并打印出来
    accuracy_score = sess.run(accuracy, feed_dict=validate_feed)
    print("After %s training step(s), validation accuracy = %g%%"
          % (global_step, accuracy_score * 100))
    # 输出After 29001 training step(s), validation accuracy = 98.62%

    # 计算在测试数据集上的准确率并打印出来
    test_accuracy = sess.run(accuracy, feed_dict=test_feed)
    print("After %s trainging step(s) ,test accuracy = %g%%"
          % (global_step, test_accuracy * 100))
    # 输出After 29001 trainging step(s) ,test accuracy = 98.51%



W1201 11:43:57.134547 139658893621056 deprecation.py:323] From <ipython-input-1-09e44113461a>:4: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1201 11:43:57.135962 139658893621056 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1201 11:43:57.137267 139658893621056 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed 

Extracting MNIST_data/train-images-idx3-ubyte.gz


W1201 11:43:57.396756 139658893621056 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W1201 11:43:57.398964 139658893621056 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W1201 11:43:57.458139 139658893621056 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.

Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


W1201 11:43:57.860744 139658893621056 deprecation.py:323] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


The latest ckpt is mnist_model.ckpt-29001
After 29001 training step(s), validation accuracy = 98.54%
After 29001 trainging step(s) ,test accuracy = 98.35%
